Apply point cloud completion (based on PoinTr) to a xyz cloud of any size. Most suitable for single trees.

In [6]:
import numpy as np
import os
import glob
import open3d as o3d
import tree2cubes


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load point cloud

In [7]:
# path to incomplete point cloud
input_file = "../files_to_test/morph_postprocessed_nbrhd_postprocessed_231_leafon_excellent_quality_classified.las.extract.ply"

In [8]:
# load incomplete pointcloud
# point_cloud = np.loadtxt(infile, skiprows=1, delimiter=" ") # delimiter=","

# load .ply file and convert to numpy array
ply_cloud = o3d.io.read_point_cloud(input_file)
point_cloud = np.asarray(ply_cloud.points)

### Cut into samples

PoinTr only allows input point clouds of a limited size. The treePoinTr models were trained on point cloud samples of 1m^3  containing between 2730 and 8192 points.
To apply completion on entire trees or even plots, larger point clouds need to be cut into cubes (voxels) to perform inference.
We use the function cut_point_cloud() to voxelize the point cloud four times with spatially shifted grids and specifyable voxel sizes. 

In [10]:
# Cut the point cloud into cubes and save as .txt files. Choose 4 cube sizes approx. between 0.6 and 1.8 m
outpath = "files_to_test_cubes/"
tree2cubes.cut_point_cloud(point_cloud, outpath, size1=1, size2=1, size3=1.25, size4=1.8)

Optional data augmentation step:
make addtional versions of the cubes where x and z are switched. 
(inference results are sometimes rotation dependent)

In [11]:
# make versions of the cubes where x and z are switched
path = "files_to_test_cubes/"

for files in glob.glob(path+"*.txt"): 
    data = np.loadtxt(files)
    filename = os.path.basename(files)
    # Swap the first and third columns
    flipfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    np.savetxt(path+filename+"_flip.txt", flipfile)

### Inference

Inference the samples with a pretrained model, following the instructions on https://github.com/yuxumin/PoinTr

For example, inference all samples under cubes/ and save the results under inference_result/, using the model real_ckpt-best.pth:



````
python tools/inference.py \
cfgs/predefhull_models/AdaPoinTr.yaml ckpts/AdaPoinTr_tree_grove_real.pth \
--pc_root "tree_workflow/files_to_test_cubes/" --save_vis_img  --save_ply \
--out_pc_root "inference_results/"

````


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


### Convert and merge predictions

In [22]:
# convert all .npy files of predictions into .ply files
pred_path="../inference_results"
dirs = os.listdir(path=pred_path)
full_pred = np.empty((2, 3))
for dirs in dirs:
    a = np.load(pred_path+"/"+dirs+"/"+"fine.npy")
    # Create Open3D point cloud and save as PLY
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(a)
    o3d.io.write_point_cloud(pred_path+"/"+dirs+"_pred.ply", cloud)
print("done")

done


In [24]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.ply"): 
    # Load PLY file using Open3D
    ply_cloud = o3d.io.read_point_cloud(files)
    data = np.asarray(ply_cloud.points)
    
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

# Save final results as PLY files
# Create and save the main completion result
cloud_pred1 = o3d.geometry.PointCloud()
cloud_pred1.points = o3d.utility.Vector3dVector(pred1)
o3d.io.write_point_cloud(pred_path+"/treename_completion.ply", cloud_pred1)

# Create and save the flipped completion result
cloud_predflip = o3d.geometry.PointCloud()
cloud_predflip.points = o3d.utility.Vector3dVector(predflip)
o3d.io.write_point_cloud(pred_path+"/treename_completion_withflips.ply", cloud_predflip)

print("done")

done


### Post-processing

Ideally, the completed point clouds are now filtered in CloudCompare, using e.g. SOR filter and Gemetric features (Surface density).